In [108]:
def f(loc, suff=None, f=None):
    with open(loc) as file:
        if f:
            return list(map(lambda l: f(l.rstrip()), file.readlines()))
        else:
            return list(map(lambda l: l.rstrip(), file.readlines()))
        
def g(day, suff=None, f=None):
    with open("/Users/maarten/Downloads/day{}.txt".format(day)) as file:
        if f:
            return list(map(lambda l: f(l.rstrip()), file.readlines()))
        else:
            return list(map(lambda l: l.rstrip(), file.readlines()))
def p(f):
    return "/Users/maarten/Downloads/{}.txt".format(f)

In [183]:
def day72(file):
    with open(file) as f:
        crabs = list(map(int, f.readline().rstrip().split(',')))
    m = None
    pos = None
    for i in range(max(crabs)):
        delta = sum(map(lambda c: abs(c-i)*(abs(c-i)+1)/2, crabs))
        if m is None or delta < m:
            m = delta
            pos = i
    return int(m)
day72(p("day7"))

100727924

In [179]:
def day7(file):
    with open(file) as f:
        crabs = list(map(int, f.readline().rstrip().split(',')))
    m = None
    pos = None
    for i in range(max(crabs)):
        delta = sum(map(lambda c: abs(c-i), crabs))
        if m is None or delta < m:
            m = delta
            pos = i
    return m
day7(p("day7"))

359648

In [165]:
def day62(file):
    fish = {key:0 for key in range(9)}
    with open(file) as f:
        l = f.readline()
        start = list(map(int, l.rstrip().split(',')))
        for f in start:
            fish[f] += 1
    for day in range(256):
        birthcount = fish[0]
        newfish = {key:0 for key in range(9)}
        # slide all fish to the left in age dict
        for age in range(9):
            newage = 6 if age == 0 else age - 1
            newfish[newage] += fish[age]
        # add the new births
        fish = newfish
        fish[8] = birthcount
    return sum(fish.values())
day62(p("day6"))

1681503251694

In [164]:
def day6(file):
    with open(file) as f:
        l = f.readline()
        fish = list(map(int, l.rstrip().split(',')))
    for day in range(80):
        newfish = []
        for i,f in enumerate(fish):
            if (f == 0):
                f = 6
                newfish.append(8)
            else:
                f -= 1
            fish[i] = f
        fish += newfish
    return len(fish)
day6(p("day6"))

372984

In [129]:
def day52(file):
    criteria = []
    hits = {}
    s = 0
    with open(file) as f:
        for line in f:
            line = line.rstrip()
            left,right = tuple(line.split('->'))
            x1,y1 = tuple(map(int, left.rstrip().split(',')))
            x2,y2 = tuple(map(int, right.rstrip().split(',')))
            xSign = 1 if x2 >= x1 else -1
            ySign = 1 if y2 >= y1 else -1
            xs = range(x1, x2+xSign, xSign)
            ys = range(y1, y2+ySign, ySign)
            if (x1 == x2 or y1 == y2):
                r = zip(list(xs)*len(ys),list(ys)*len(xs))
            else:
                r = zip(list(xs),list(ys))
            for x,y in r:
                if (x,y) in hits:
                    if hits[(x,y)] == 1:
                        s += 1
                    hits[(x,y)] += 1
                else:
                    hits[(x,y)] = 1
    return s
day52(p("day5"))

15463

In [109]:
def day5(file):
    criteria = []
    hits = {}
    s = 0
    with open(file) as f:
        for line in f:
            line = line.rstrip()
            left,right = tuple(line.split('->'))
            x1,y1 = tuple(map(int, left.rstrip().split(',')))
            x2,y2 = tuple(map(int, right.rstrip().split(',')))
            if not (x1 == x2 or y1 == y2):
                continue
            for x in range(min(x1, x2), max(x1, x2)+1):
                for y in range(min(y1, y2), max(y1, y2)+1):
                    if (x,y) in hits:
                        if hits[(x,y)] == 1:
                            s += 1
                        hits[(x,y)] += 1
                    else:
                        hits[(x,y)] = 1
                    
    return s
day5(p("day5"))

5698

In [87]:
def board_mark(board, number):
    for l in board:
        if number in l:
            idx = l.index(number)
            l[idx] = -10000
    return board

def score_board(board):
    s = 0
    for l in board:
        s += sum(filter(lambda n: n > 0, l))
    return s

def val_board(board):
    neg_row = filter(lambda l: all(map(lambda n: n<0, l)), board)
    neg_col = filter(lambda l: all(map(lambda n: n<0, l)), list(map(list, zip(*board))))
    return any(neg_row) or any(neg_col)
                     
def day4_2(file):
    with open(file) as f:
        draws = map(int, f.readline().split(","))
        
        boards = []
        done = set([])
        board = None
        for line in f:
            line = line.rstrip()
            if not line:
                if board:
                    boards.append(board)
                board = []
            else:
                board.append(list(map(int, line.split())))
        boards.append(board)
        
        for draw in draws:
            for i, board in enumerate(boards):
                board = board_mark(board, draw)
                if val_board(board):
                    done.add(i)
                    if len(done) == len(boards):
                        s = score_board(board)
                        return s*draw
                boards[i] = board
print(day4_2("/Users/maarten/Downloads/day4.txt"))

21070


In [75]:
def board_mark(board, number):
    for l in board:
        if number in l:
            idx = l.index(number)
            l[idx] = -10000
    return board

def score_board(board):
    s = 0
    for l in board:
        s += sum(filter(lambda n: n > 0, l))
    return s

def val_board(board):
    neg_row = filter(lambda l: all(map(lambda n: n<0, l)), board)
    neg_col = filter(lambda l: all(map(lambda n: n<0, l)), list(map(list, zip(*board))))
    return any(neg_row) or any(neg_col)
                     
def day4(file):
    with open(file) as f:
        draws = map(int, f.readline().split(","))
        
        boards = []
        board = None
        for line in f:
            line = line.rstrip()
            if not line:
                if board:
                    boards.append(board)
                board = []
            else:
                board.append(list(map(int, line.split())))
        boards.append(board)
        
        for draw in draws:
            for i, board in enumerate(boards):
                board = board_mark(board, draw)
                if val_board(board):
                    s = score_board(board)
                    return s*draw
                boards[i] = board
print(day4("/Users/maarten/Downloads/demo.txt"))

4512


In [8]:
# Day 3.2
def day3(file):
    lines = f(file)
    bits = {}
    
    mkeep = lines
    lkeep = lines
    m = 0
    l = 0
    for i in range(len(lines[0])):
        s = sum(map(lambda l: int(l[i]), mkeep))
        if s >= len(mkeep)/2:
            mkeep = list(filter(lambda l: l[i] == '1', mkeep))
        else:
            mkeep = list(filter(lambda l: l[i] == '0', mkeep))
        if len(mkeep) == 1:
            m = int(min(mkeep), 2)
        s = sum(map(lambda l: int(l[i]), lkeep))
        if s >= len(lkeep)/2 and s != len(lkeep):
            lkeep = list(filter(lambda l: l[i] == '0', lkeep))
        else:
            lkeep = list(filter(lambda l: l[i] == '1', lkeep))
        if len(lkeep) == 1:
            l = int(min(lkeep), 2)
    return m*l
        
day3("/Users/maarten/Downloads/report18.txt".format(3))

43774224

In [4]:
# Day 3.1
def day3():
    lines = f("/Users/maarten/Downloads/day{}.txt".format(3))
    bits = {}
    n = 0

    for line in lines:
        for i,bit in enumerate(line):
            if i in bits:
                bits[i]+=int(line[i])
            else:
                bits[i]=int(line[i])
        n+=1
        
    m = ""
    l = ""

    for k,v in sorted(bits.items(), key = lambda d: d[0]):
        if v<n/2:
            m += "0"
            l += "1"
        else:
            m += "1"
            l += "0"
    return int(m, 2)*int(l,2)
day3()

1307354

In [43]:
day2 = g(2)

In [44]:
#Day 2
deltas = {"forward": (1,0), "down": (0,1), "up": (0,-1)}
pos = (0,0)
for d in day2:
    inst, amount = tuple(d.split(" "))
    amount = int(amount)
    delta = deltas[inst]
    pos = (pos[0] + amount * delta[0], pos[1] + amount * delta[1])
pos[0]*pos[1]

2272262

In [41]:
#Day 2.2
aim = 0
hpos = 0
depth = 0
for d in day2:
    inst, amount = tuple(d.split(" "))
    amount = int(amount)
    if inst == "down":
        aim += amount
    if inst == "up":
        aim -= amount
    if inst == "forward":
        hpos += amount
        depth += aim * amount
hpos * depth

2134882034

In [31]:
day1 = g(1, f=int)

In [33]:
#Day 1
prev = None
cnt = 0
for d in day1:
    if prev and d > prev:
        cnt += 1
    prev = d
cnt

1390

In [35]:
#Day 1.2
prev = None
cnt = 0
for i,d in enumerate(day1):
    s = sum(day1[i:i+3])
    if prev and s > prev:
        cnt += 1
    prev = s
cnt

1457